## Python Tools for Record Linking and Fuzzy Matching

This notebook accompanies the [article](https://pbpython.com/record-linking.html) on Practical Business Python

This notebook relies on [fuzzymatcher](https://github.com/RobinL/fuzzymatcher) and the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/about.html)


## first install the required packages

In [9]:
!pip install fuzzymatcher

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.2 MB 13.6 MB/s 
     |████████████████████████████████| 170 kB 94.0 MB/s 
  Created wheel for metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13918 sha256=4bbf79de9a775e3d1c4888fdaa8f7f2aa419a6ba8fa7863fcffa6d54c5991d95
  Stored in directory: /root/.cache/pip/wheels/1d/a8/cb/6f8902aa5457bd71344e00665c230e9c45255b3f57f2194a0f
Successfully built metaphone


In [1]:
pip install splink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 14.2 MB/s 
     |████████████████████████████████| 104 kB 67.1 MB/s 
     |████████████████████████████████| 133 kB 75.1 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 16.4 MB 43.2 MB/s 
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.3.3
    Uninstalling jsonschema-4.3.3:
      Successfully uninstalled jsonschema-4.3.3
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.


In [10]:
!pip install recordlinkage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 926 kB 12.8 MB/s 
     |████████████████████████████████| 132 kB 69.8 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=74015 sha256=28277a2dd6abf894d0c40b5f553e5e5a595b02851b68514ceaf17ad9e40ec1b0
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


## standard imports 

In [5]:
import pandas as pd
from pathlib import Path

## data load is the same for each package

In [15]:
hospital_accounts = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_info.csv'
)
hospital_reimbursement = pd.read_csv(
    'https://raw.githubusercontent.com/chris1610/pbpython/master/data/hospital_reimbursement.csv'
)

In [16]:
hospital_accounts.head()

,Account_Num,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
0,10605,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private
1,24250,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary
2,10341,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary
4,39835,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary


In [17]:
hospital_reimbursement.head()

,Provider_Num,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,118,20855.61,5026.19,4115.52
1,519118,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,43,13289.09,5413.63,4490.93
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,73,22261.60,4922.18,4021.79
3,201752,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,12,10901.33,5343.50,4284.17
4,678488,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,74,28117.95,5947.12,4819.53


In [18]:
## dataframe sizes

print("raw dataframe number of records:")
print("hospital_accounts:  {x}".format(x=hospital_accounts['Account_Num'].count()))
print("hospital_reimbursement:  {x}".format(x=hospital_reimbursement['Provider_Num'].count()))

raw dataframe number of records:
hospital_accounts:  5339
hospital_reimbursement:  2697


### first we just explore the data and then try exact joins

#### focus on "Facility Name" and "Provider Name"
 just pick providers starting with 'A' from both, sort and pick top 20 to visually compare


In [ ]:

cond = hospital_accounts['Facility Name'].str.startswith('AD')
hospital_accounts.loc[cond].sort_values(by=['Facility Name'], ascending=True)[['Facility Name', 'Address']].head(25)

,Facility Name,Address
3282,"AD HOSPITAL EAST, LLC","12950 EAST FREEWAY, SUITE 100"
736,ADAIR ACUTE CARE AT OSAWATOMIE STATE HOSPITAL,500 STATE HOSPITAL DRIVE
2515,ADAIR COUNTY MEMORIAL HOSPITAL,609 SE KENT
3915,ADAMS COUNTY REGIONAL MEDICAL CENTER,230 MEDICAL CENTER DRIVE
280,ADAMS MEMORIAL HOSPITAL,1100 MERCER AVE
3017,ADCARE HOSPITAL OF WORCESTER INC,107 LINCOLN STREET
2883,ADENA PIKE MEDICAL CENTER,100 DAWN LANE
4211,ADENA REGIONAL MEDICAL CENTER,272 HOSPITAL ROAD
4887,ADIRONDACK MEDICAL CENTER - SARANAC LAKE,"2233 STATE ROUTE 86, PO BOX 471"
4429,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,BO MONACILLO CARR NUM 22


In [ ]:
cond = hospital_reimbursement['Provider Name'].str.startswith('AD')
hospital_reimbursement.sort_values(by=['Provider Name'], ascending=True)[['Provider Name','Provider Street Address', 'Provider City']].head(25)


,Provider Name,Provider Street Address,Provider City
1051,ABBEVILLE GENERAL HOSPITAL,118 N HOSPITAL DR,ABBEVILLE
1306,ABBOTT NORTHWESTERN HOSPITAL,800 EAST 28TH STREET,MINNEAPOLIS
2376,ABILENE REGIONAL MEDICAL CENTER,6250 HWY 83/84,ABILENE
2109,ABINGTON MEMORIAL HOSPITAL,1200 OLD YORK ROAD,ABINGTON
99,ABRAZO ARROWHEAD CAMPUS,18701 NORTH 67TH AVENUE,GLENDALE
91,ABRAZO SCOTTSDALE CAMPUS,3929 EAST BELL ROAD,PHOENIX
103,ABRAZO WEST CAMPUS,13677 WEST MCDOWELL ROAD,GOODYEAR
1056,ACADIA GENERAL HOSPITAL,1305 CROWLEY RAYNE HIGHWAY,CROWLEY
2080,ACMH HOSPITAL,ONE NOLTE DRIVE,KITTANNING
1883,ADENA REGIONAL MEDICAL CENTER,272 HOSPITAL ROAD,CHILLICOTHE


### try first a straight left outer join on the name

see https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


In [ ]:

merged = hospital_accounts.merge(hospital_reimbursement, left_on = 'Facility Name', right_on='Provider Name',  how='left')

## show the records where on the left we have the AD hospitals
cond = merged['Facility Name'].str.startswith('AD')
merged[cond][['Facility Name', 'Provider Name']].sort_values(by='Facility Name').head(20)


,Facility Name,Provider Name
3519,"AD HOSPITAL EAST, LLC",NaN
784,ADAIR ACUTE CARE AT OSAWATOMIE STATE HOSPITAL,NaN
2704,ADAIR COUNTY MEMORIAL HOSPITAL,NaN
4213,ADAMS COUNTY REGIONAL MEDICAL CENTER,NaN
307,ADAMS MEMORIAL HOSPITAL,NaN
3246,ADCARE HOSPITAL OF WORCESTER INC,NaN
3102,ADENA PIKE MEDICAL CENTER,NaN
4527,ADENA REGIONAL MEDICAL CENTER,ADENA REGIONAL MEDICAL CENTER
5245,ADIRONDACK MEDICAL CENTER - SARANAC LAKE,NaN
4764,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,NaN


In [ ]:
## how many have we matched?
merged[merged['Provider Name'].notnull()]['Provider Name'].count()

2739

### how about a full outer join?

In [ ]:
merged = hospital_accounts.merge(hospital_reimbursement, left_on = 'Facility Name', right_on='Provider Name',  how='outer')

## show the records where on the left we have the AD hospitals
cond1 = (merged['Facility Name'].notnull()) 
cond2 = (merged['Facility Name'].str.startswith('AD'))
merged[cond1 & cond2][['Facility Name', 'Provider Name']].sort_values(by='Facility Name').head(20)


,Facility Name,Provider Name
3757,"AD HOSPITAL EAST, LLC",NaN
1029,ADAIR ACUTE CARE AT OSAWATOMIE STATE HOSPITAL,NaN
2971,ADAIR COUNTY MEMORIAL HOSPITAL,NaN
4367,ADAMS COUNTY REGIONAL MEDICAL CENTER,NaN
438,ADAMS MEMORIAL HOSPITAL,NaN
3500,ADCARE HOSPITAL OF WORCESTER INC,NaN
3370,ADENA PIKE MEDICAL CENTER,NaN
4652,ADENA REGIONAL MEDICAL CENTER,ADENA REGIONAL MEDICAL CENTER
5301,ADIRONDACK MEDICAL CENTER - SARANAC LAKE,NaN
4862,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,NaN


## Example using fuzzymatcher

In [12]:
import fuzzymatcher

In [ ]:
# Columns to match on from df_left
left_on = ["Facility Name", "Address", "City", "State"]

# Columns to match on from df_right
right_on = [    "Provider Name",  "Provider Street Address", "Provider City",    "Provider State"]

In [ ]:
## try link_table first
linked = fuzzymatcher.link_table(hospital_accounts,
                                               hospital_reimbursement,
                                               left_on,
                                               right_on,
                                               left_id_col='Account_Num',
                                               right_id_col='Provider_Num')

In [ ]:
linked.head()

,__id_left,__id_right,match_score,match_rank,Facility Name,Provider Name
0,10605,741329.0,-0.024854,1,SAGE MEMORIAL HOSPITAL,MEMORIAL REGIONAL HOSPITAL
1,10605,730311.0,-0.074309,2,SAGE MEMORIAL HOSPITAL,JACKSON MEMORIAL HOSPITAL
2,10605,657875.0,-0.077214,3,SAGE MEMORIAL HOSPITAL,JOHNSON MEMORIAL HOSPITAL
3,10605,299065.0,-0.085724,4,SAGE MEMORIAL HOSPITAL,HUNTINGTON MEMORIAL HOSPITAL
4,10605,895820.0,-0.085724,5,SAGE MEMORIAL HOSPITAL,LAWRENCE & MEMORIAL HOSPITAL


In [ ]:
linked[linked['Facility Name'].str.contains('GORDON')].sort_values(by='match_score', ascending=False)

,__id_left,__id_right,match_score,match_rank,Facility Name,Provider Name
83533,24125,515652.0,0.117880,1,ADVENTHEALTH GORDON,GORDON HOSPITAL
83381,24261,726996.0,0.100981,1,GORDON MEMORIAL HOSPITAL DISTRICT,MEMORIAL HOSPITAL OF GARDENA
83382,24261,741329.0,-0.128907,2,GORDON MEMORIAL HOSPITAL DISTRICT,MEMORIAL REGIONAL HOSPITAL
83383,24261,730311.0,-0.178362,3,GORDON MEMORIAL HOSPITAL DISTRICT,JACKSON MEMORIAL HOSPITAL
83384,24261,657875.0,-0.181267,4,GORDON MEMORIAL HOSPITAL DISTRICT,JOHNSON MEMORIAL HOSPITAL
83385,24261,299065.0,-0.189776,5,GORDON MEMORIAL HOSPITAL DISTRICT,HUNTINGTON MEMORIAL HOSPITAL
83386,24261,895820.0,-0.189776,6,GORDON MEMORIAL HOSPITAL DISTRICT,LAWRENCE & MEMORIAL HOSPITAL
83387,24261,308278.0,-0.193006,7,GORDON MEMORIAL HOSPITAL DISTRICT,LEE MEMORIAL HOSPITAL
83392,24261,220714.0,-0.203041,12,GORDON MEMORIAL HOSPITAL DISTRICT,WOODLAND MEMORIAL HOSPITAL
83393,24261,558189.0,-0.203041,13,GORDON MEMORIAL HOSPITAL DISTRICT,CITRUS MEMORIAL HOSPITAL


In [ ]:
# Now perform the match
# It will take several minutes to run on this data set
matched_results = fuzzymatcher.fuzzy_left_join(hospital_accounts,
                                               hospital_reimbursement,
                                               left_on,
                                               right_on,
                                               left_id_col='Account_Num',
                                               right_id_col='Provider_Num')

In [ ]:
matched_results[["Facility Name", "Provider Name"]].head()

,Facility Name,Provider Name
0,SAGE MEMORIAL HOSPITAL,TYLER MEMORIAL HOSPITAL
234,WOODRIDGE BEHAVIORAL CENTER,CRISP REGIONAL HOSPITAL
446,DOUGLAS GARDENS HOSPITAL,JACKSON MEMORIAL HOSPITAL
675,SUNCOAST BEHAVIORAL HEALTH CENTER,BLAKE MEDICAL CENTER
893,TREASURE VALLEY HOSPITAL,ST LUKE'S REGIONAL MEDICAL CENTER


In [ ]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "Facility Name", "Provider Name", "Address", "Provider Street Address",
    "Provider City", "City", "Provider State", "State"
]

In [ ]:
# Let's see the best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
77995,3.090931,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,RARITAN BAY MEDICAL CENTER PERTH AMBOY DIVISION,530 NEW BRUNSWICK AVE,530 NEW BRUNSWICK AVE,PERTH AMBOY,PERTH AMBOY,NJ,NJ
532606,2.799072,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ROBERT WOOD JOHNSON UNIVERSITY HOSPITAL,ONE ROBERT WOOD JOHNSON PLACE,ONE ROBERT WOOD JOHNSON PLACE,NEW BRUNSWICK,NEW BRUNSWICK,NJ,NJ
78605,2.785132,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,AVERA MCKENNAN HOSPITAL & UNIVERSITY HEALTH CE...,1325 S CLIFF AVE POST OFFICE BOX 5045,1325 S CLIFF AVE POST OFFICE BOX 5045,SIOUX FALLS,SIOUX FALLS,SD,SD
242528,2.778860,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,JOHN T MATHER MEMORIAL HOSPITAL OF PORT JEFFE...,75 NORTH COUNTRY ROAD,75 NORTH COUNTRY ROAD,PORT JEFFERSON,PORT JEFFERSON,NY,NY
447173,2.721425,MAYO CLINIC HEALTH SYSTEM - RED WING,MAYO CLINIC HEALTH SYSTEM IN RED WING,"701 HEWITT BOULEVARD, PO BOX 95","701 HEWITT BOULEVARD, PO BOX 95",RED WING,RED WING,MN,MN


In [ ]:
# Now the worst matches
matched_results[cols].sort_values(by=['best_match_score'],
                                  ascending=True).head(5)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
427229,-2.268231,CENTRO MEDICO WILMA N VAZQUEZ,BAPTIST MEDICAL CENTER EAST,CARR. 2 KM 39.5 ROAD NUMBER 2 BO ALGARROBO,400 TAYLOR ROAD,MONTGOMERY,VEGA BAJA,AL,PR
83304,-2.124071,DOCTOR CENTER HOSPITAL SAN FERNANDO DE LA CARO...,OVERLAKE HOSPITAL MEDICAL CENTER,EDIF JESUS T PINEIRO AVE FERNANDEZ JUNCOS BO P...,1035-116TH AVE NE,BELLEVUE,CAROLINA,WA,PR
42761,-2.106746,HOSPITAL ONCOLOGICO DR ISAAC GONZALEZ MARTINEZ,SCRIPPS MERCY HOSPITAL,BO. MONACILLOS CARR 22 CENTRO MEDICO DE PUERTO...,4077 5TH AVE,SAN DIEGO,SAN JUAN,CA,PR
451126,-2.050888,CENTRO DE SALUD CONDUCTUAL MENONITA-CIMA,MILFORD REGIONAL MEDICAL CENTER,CARR ESTATAL 14 INTERIOR SARGENTO GERARDO SANT...,14 PROSPECT STREET,MILFORD,AIBONITO,MA,PR
477033,-1.996508,ADMIN DE SERVICIOS MEDICOS PUERTO RIC,MAINE MEDICAL CENTER,BO MONACILLO CARR NUM 22,22 BRAMHALL ST,PORTLAND,SAN JUAN,ME,PR


In [ ]:
# Look at the matches around 1
matched_results[cols].sort_values(
    by=['best_match_score'], ascending=False).head(10)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
414728,0.999843,PRISMA HEALTH BAPTIST,PALMETTO HEALTH BAPTIST,1330 TAYLOR AT MARION ST,TAYLOR AT MARION ST,COLUMBIA,COLUMBIA,SC,SC
148944,0.999724,CHRIST HOSPITAL,CHRIST HOSPITAL,2139 AUBURN AVENUE,2139 AUBURN AVENUE,CINCINNATI,CINCINNATI,OH,OH
388721,0.997759,RUSSELL COUNTY HOSPITAL,RUSSELL COUNTY MEDICAL CENTER,58 CARROLL STREET,58 CARROLL STREET,LEBANON,LEBANON,VA,VA
378541,0.993306,SKY RIDGE MEDICAL CENTER,SKY RIDGE MEDICAL CENTER,10101 RIDGEGATE PARKWAY,10101 RIDGE GATE PARKWAY,LONE TREE,LONE TREE,CO,CO
304062,0.991168,HIGHLAND HOSPITAL,HIGHLAND HOSPITAL,1000 SOUTH AVENUE,1000 SOUTH AVENUE,ROCHESTER,ROCHESTER,NY,NY
539270,0.990481,PAOLI HOSPITAL,MAIN LINE HOSPITAL PAOLI,255 WEST LANCASTER AVENUE,255 WEST LANCASTER AVENUE,PAOLI,PAOLI,PA,PA
251923,0.986695,NYU WINTHROP HOSPITAL,WINTHROP-UNIVERSITY HOSPITAL,259 FIRST STREET,259 FIRST STREET,MINEOLA,MINEOLA,NY,NY
311394,0.986440,ADVENTHEALTH GORDON,GORDON HOSPITAL,1035 RED BUD ROAD,1035 RED BUD ROAD,CALHOUN,CALHOUN,GA,GA
204356,0.985763,MEMORIAL MEDICAL CENTER,MEMORIAL MEDICAL CENTER,701 N FIRST ST,701 N FIRST ST,SPRINGFIELD,SPRINGFIELD,IL,IL
102416,0.984978,HENDRICK MEDICAL CENTER,HENDRICK MEDICAL CENTER,1900 PINE,1900 PINE,ABILENE,ABILENE,TX,TX


In [ ]:
matched_results[cols].query("best_match_score <= .6").sort_values(
    by=['best_match_score'], ascending=False).head(5)

,best_match_score,Facility Name,Provider Name,Address,Provider Street Address,Provider City,City,Provider State,State
9976,0.594255,ADVENTHEALTH CENTRAL TEXAS,METROPLEX HOSPITAL,2201 S CLEAR CREEK ROAD,2201 S CLEAR CREEK ROAD,KILLEEN,KILLEEN,TX,TX
158487,0.585892,HOUSTON METHODIST THE WOODLANDS HOSPITAL,ST LUKE'S THE WOODLANDS HOSPITAL,17201 INTERSTATE 45 SOUTH,17200 ST LUKE'S WAY,THE WOODLANDS,THE WOODLANDS,TX,TX
489328,0.576772,MUSC HEALTH MARION MEDICAL CENTER,CAROLINAS HOSPITAL SYSTEM MARION,2829 E HWY 76,2829 E HWY 76,MULLINS,MULLINS,SC,SC
466350,0.575092,MERCY HEALTH-LORAIN HOSPITAL,MERCY REGIONAL MEDICAL CENTER,3700 KOLBE ROAD,3700 KOLBE ROAD,LORAIN,LORAIN,OH,OH
377734,0.566592,UT HEALTH EAST TEXAS TYLER REGIONAL HOSPITAL,EAST TEXAS MEDICAL CENTER,1000 SOUTH BECKHAM AVE,1000 SOUTH BECKHAM STREET,TYLER,TYLER,TX,TX


## Example using Splink

In [2]:
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.duckdb.duckdb_comparison_library import (
    exact_match,
    levenshtein_at_thresholds,
)

In [23]:
hospital_dupes = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_dupes.csv',
    index_col='Account_Num')

In [24]:
settings = {
    "link_type": "dedupe_only",
    "comparisons": [
        levenshtein_at_thresholds("Facility Name", 2)
    ],
}


In [25]:
linker = DuckDBLinker(hospital_dupes, settings)


In [26]:
pairwise_predictions = linker.predict()

BinderException: ignored

## Example using Python Record Linkage Toolkit

In [19]:
import recordlinkage

### first repeat the analysis we did using the fuzzyMatcher
what additional parameters can we set?
do we get different results?

In [ ]:
# Build the indexer
indexer = recordlinkage.Index()
# Can use full or block
# indexer.full()
indexer.block(left_on='State', right_on='Provider State')

# Use sortedneighbor as a good option if data is not clean
# indexer.sortedneighbourhood(left_on='State', right_on='Provider State')

<Index>

In [ ]:
candidates = indexer.index(hospital_accounts, hospital_reimbursement)

In [ ]:
# Let's see how many matches we want to do
print(len(candidates))

475830


In [ ]:
# Takes 3 minutes using the full index.
# 14s using sorted neighbor
# 7s using blocking
compare = recordlinkage.Compare()
compare.exact('City', 'Provider City', label='City')
compare.string('Facility Name',
               'Provider Name',
               threshold=0.85,
               label='Hosp_Name')
compare.string('Address',
               'Provider Street Address',
               method='jarowinkler',
               threshold=0.85,
               label='Hosp_Address')
features = compare.compute(candidates, hospital_accounts,
                           hospital_reimbursement)

In [ ]:
features.describe()

,City,Hosp_Name,Hosp_Address
count,475830.000000,475830.000000,475830.000000
mean,0.019372,0.005294,0.007431
std,0.137830,0.072566,0.085884
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


In [ ]:
# What are the score totals?
features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0      2290
2.0       461
1.0      8261
0.0    987848
dtype: int64

In [ ]:
# Get the potential matches
potential_matches = features[features.sum(axis=1) > 1].reset_index()

In [ ]:
potential_matches['Score'] = potential_matches.loc[:, 'City':'Hosp_Address'].sum(axis=1)
potential_matches.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score
0,51216,268781,0,1.0,1.0,2.0
1,55272,556917,1,1.0,1.0,3.0
2,87807,854637,1,1.0,1.0,3.0
3,51151,783146,1,0.0,1.0,2.0
4,11740,260374,1,1.0,1.0,3.0


In [ ]:
hospital_accounts.loc[51216,:]

Facility Name             ST FRANCIS MEDICAL CENTER
Address                       2400 ST FRANCIS DRIVE
City                                   BRECKENRIDGE
State                                            MN
ZIP Code                                      56520
County Name                                  WILKIN
Phone Number                         (218) 643-3000
Hospital Type             Critical Access Hospitals
Hospital Ownership    Voluntary non-profit - Church
Name: 51216, dtype: object

In [ ]:
hospital_reimbursement.loc[268781,:]

Provider Name                SAINT FRANCIS MEDICAL CENTER
Provider Street Address                 211 ST FRANCIS DR
Provider City                              CAPE GIRARDEAU
Provider State                                         MO
Provider Zip Code                                   63703
Total Discharges                                      141
Average Covered Charges                          42515.09
Average Total Payments                            5902.24
Average Medicare Payments                         4993.43
Name: 268781, dtype: object

In [ ]:
# Add some convenience columns for comparing data
hospital_accounts['Acct_Name_Lookup'] = hospital_accounts[[
    'Facility Name', 'Address', 'City', 'State'
]].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
hospital_reimbursement['Reimbursement_Name_Lookup'] = hospital_reimbursement[[
    'Provider Name', 'Provider Street Address', 'Provider City',
    'Provider State'
]].apply(lambda x: '_'.join(x), axis=1)

In [ ]:
reimbursement_lookup = hospital_reimbursement[['Reimbursement_Name_Lookup']].reset_index()
account_lookup = hospital_accounts[['Acct_Name_Lookup']].reset_index()

In [ ]:
account_lookup.head()

,Account_Num,Acct_Name_Lookup
0,10605,SAGE MEMORIAL HOSPITAL_STATE ROUTE 264 SOUTH 1...
1,24250,WOODRIDGE BEHAVIORAL CENTER_600 NORTH 7TH STRE...
2,10341,DOUGLAS GARDENS HOSPITAL_5200 NE 2ND AVE_MIAMI_FL
3,81101,SUNCOAST BEHAVIORAL HEALTH CENTER_4480 51ST ST...
4,39835,TREASURE VALLEY HOSPITAL_8800 WEST EMERALD STR...


In [ ]:
reimbursement_lookup.head()

,Provider_Num,Reimbursement_Name_Lookup
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER_1108 ROSS CLA...
1,519118,MARSHALL MEDICAL CENTER SOUTH_2505 U S HIGHWAY...
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL_205 MARENGO STR...
3,201752,MIZELL MEMORIAL HOSPITAL_702 N MAIN ST_OPP_AL
4,678488,ST VINCENT'S EAST_50 MEDICAL PARK EAST DRIVE_B...


In [ ]:
account_merge = potential_matches.merge(account_lookup, how='left')

In [ ]:
account_merge.head()

,Account_Num,Provider_Num,City,Hosp_Name,Hosp_Address,Score,Acct_Name_Lookup
0,51216,268781,0,1.0,1.0,2.0,ST FRANCIS MEDICAL CENTER_2400 ST FRANCIS DRIV...
1,55272,556917,1,1.0,1.0,3.0,SCOTTSDALE OSBORN MEDICAL CENTER_7400 EAST OSB...
2,87807,854637,1,1.0,1.0,3.0,ORO VALLEY HOSPITAL_1551 EAST TANGERINE ROAD_O...
3,51151,783146,1,0.0,1.0,2.0,"ST. LUKE'S BEHAVIORAL HOSPITAL, LP_1800 EAST V..."
4,11740,260374,1,1.0,1.0,3.0,SUMMIT HEALTHCARE REGIONAL MEDICAL CENTER_2200...


In [ ]:
reimbursement_lookup.head()

,Provider_Num,Reimbursement_Name_Lookup
0,839987,SOUTHEAST ALABAMA MEDICAL CENTER_1108 ROSS CLA...
1,519118,MARSHALL MEDICAL CENTER SOUTH_2505 U S HIGHWAY...
2,733073,ELIZA COFFEE MEMORIAL HOSPITAL_205 MARENGO STR...
3,201752,MIZELL MEMORIAL HOSPITAL_702 N MAIN ST_OPP_AL
4,678488,ST VINCENT'S EAST_50 MEDICAL PARK EAST DRIVE_B...


In [ ]:
# Let's build a dataframe to  compare
final_merge = account_merge.merge(reimbursement_lookup, how='left')

In [ ]:
cols = [
    'Account_Num', 'Provider_Num', 'Score', 'Acct_Name_Lookup',
    'Reimbursement_Name_Lookup'
]

In [ ]:
final_merge[cols].sort_values(by=['Account_Num', 'Score'], ascending=False)

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
2675,94995,825914,3.0,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...,CLAIBORNE MEDICAL CENTER_1850 OLD KNOXVILLE HI...
1987,94953,819181,3.0,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...,LAKE CHARLES MEMORIAL HOSPITAL_1701 OAK PARK B...
1045,94943,680596,3.0,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...,VALLEY PRESBYTERIAN HOSPITAL_15107 VANOWEN ST_...
2319,94923,403151,3.0,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...,UNIVERSITY COLO HEALTH MEMORIAL HOSPITAL CENTR...
2526,94887,752284,2.0,NEW YORK-PRESBYTERIAN BROOKLYN METHODIST HOSPI...,NEW YORK METHODIST HOSPITAL_506 SIXTH STREET_B...
...,...,...,...,...,...
2093,10165,188247,3.0,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT,UTAH VALLEY HOSPITAL_1034 NORTH 500 WEST_PROVO_UT
1836,10090,212069,3.0,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...,CANONSBURG GENERAL HOSPITAL_100 MEDICAL BOULEV...
2438,10043,140535,3.0,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...,BETH ISRAEL DEACONESS HOSPITAL - PLYMOUTH_275 ...
1971,10020,210657,3.0,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...,ST FRANCIS MEDICAL CENTER_309 JACKSON STREET_M...


In [ ]:
# If you need to save it to Excel -
#final_merge.sort_values(by=['Account_Num', 'Score'],
#                        ascending=False).to_excel('merge_list.xlsx',
#                                                  index=False)

In [ ]:
final_merge[final_merge['Account_Num']==11035][cols]

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
2316,11035,550921,2.0,SAINT JOSEPH HOSPITAL_1375 E 19TH AVE_DENVER_CO,PRESBYTERIAN ST LUKES MEDICAL CENTER_1719 E 19...
2317,11035,706807,3.0,SAINT JOSEPH HOSPITAL_1375 E 19TH AVE_DENVER_CO,SAINT JOSEPH HOSPITAL_1375 EAST 19TH AVE_DENVE...


In [ ]:
final_merge[final_merge['Account_Num']==56375][cols]

,Account_Num,Provider_Num,Score,Acct_Name_Lookup,Reimbursement_Name_Lookup
155,56375,390402,2.0,BROWARD HEALTH IMPERIAL POINT_6401 N FEDERAL H...,HOLY CROSS HOSPITAL_4725 N FEDERAL HWY_FORT LA...
156,56375,451229,3.0,BROWARD HEALTH IMPERIAL POINT_6401 N FEDERAL H...,BROWARD HEALTH IMPERIAL POINT_6401 N FEDERAL H...


### then dedupe the data, as this package allows this

In [ ]:
hospital_dupes = pd.read_csv(
    'https://github.com/chris1610/pbpython/raw/master/data/hospital_account_dupes.csv',
    index_col='Account_Num')

In [ ]:
hospital_dupes.head()

,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
Account_Num,,,,,,,,,
71730,SAGE MEMORIAL HOSPITAL,STATE ROUTE 264 SOUTH 191,GANADO,AZ,86505,APACHE,(928) 755-4541,Critical Access Hospitals,Voluntary non-profit - Private
70116,WOODRIDGE BEHAVIORAL CENTER,600 NORTH 7TH STREET,WEST MEMPHIS,AR,72301,CRITTENDEN,(870) 394-4113,Psychiatric,Proprietary
87991,DOUGLAS GARDENS HOSPITAL,5200 NE 2ND AVE,MIAMI,FL,33137,MIAMI-DADE,(305) 751-8626,Acute Care Hospitals,Voluntary non-profit - Private
22662,SUNCOAST BEHAVIORAL HEALTH CENTER,4480 51ST ST W,BRADENTON,FL,34210,MANATEE,(941) 792-2222,Psychiatric,Proprietary
63165,TREASURE VALLEY HOSPITAL,8800 WEST EMERALD STREET,BOISE,ID,83704,ADA,(208) 373-5000,Acute Care Hospitals,Proprietary


In [ ]:
# Deduping follows the same process, you just use 1 single dataframe
dupe_indexer = recordlinkage.Index()
dupe_indexer.sortedneighbourhood(left_on='State')
dupe_candidate_links = dupe_indexer.index(hospital_dupes)


In [ ]:
# Comparison step
compare_dupes = recordlinkage.Compare()
compare_dupes.string('City', 'City', threshold=0.85, label='City')
compare_dupes.string('Phone Number',
                     'Phone Number',
                     threshold=0.85,
                     label='Phone_Num')
compare_dupes.string('Facility Name',
                     'Facility Name',
                     threshold=0.80,
                     label='Hosp_Name')
compare_dupes.string('Address',
                     'Address',
                     threshold=0.85,
                     label='Hosp_Address')
dupe_features = compare_dupes.compute(dupe_candidate_links, hospital_dupes)

In [ ]:
dupe_features

City  Phone_Num  Hosp_Name  Hosp_Address
Account_Num_1 Account_Num_2                                          
26270         28485           0.0        0.0        0.0           0.0
              30430           0.0        0.0        0.0           0.0
              43602           0.0        0.0        0.0           0.0
59585         28485           0.0        0.0        0.0           0.0
              30430           0.0        0.0        0.0           0.0
...                           ...        ...        ...           ...
64029         38600           0.0        0.0        0.0           0.0
              35413           0.0        0.0        0.0           0.0
              81525           0.0        0.0        0.0           0.0
              82916           0.0        0.0        0.0           0.0
              18907           0.0        0.0        0.0           0.0

[981277 rows x 4 columns]

In [ ]:
dupe_features.sum(axis=1).value_counts().sort_index(ascending=False)

3.0         7
2.0       206
1.0      7859
0.0    973205
dtype: int64

In [ ]:
potential_dupes = dupe_features[dupe_features.sum(axis=1) > 2].reset_index()
potential_dupes['Score'] = potential_dupes.loc[:, 'City':'Hosp_Address'].sum(axis=1)

In [ ]:
potential_dupes.sort_values(by=['Score'], ascending=True)

,Account_Num_1,Account_Num_2,City,Phone_Num,Hosp_Name,Hosp_Address,Score
0,28494,37949,1.0,1.0,0.0,1.0,3.0
1,74835,77000,1.0,1.0,0.0,1.0,3.0
2,24549,28485,1.0,1.0,0.0,1.0,3.0
3,70366,52654,1.0,1.0,0.0,1.0,3.0
4,61685,24849,1.0,1.0,0.0,1.0,3.0
5,51567,41166,1.0,1.0,1.0,0.0,3.0
6,26495,41079,1.0,1.0,0.0,1.0,3.0


In [ ]:
# Take a look at one of the potential duplicates
hospital_dupes[hospital_dupes.index.isin([51567, 41166])]

,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership
Account_Num,,,,,,,,,
41166,ST VINCENT HOSPITAL,835 S VAN BUREN ST,GREEN BAY,WI,54301,BROWN,(920) 433-0111,Acute Care Hospitals,Voluntary non-profit - Church
51567,SAINT VINCENT HOSPITAL,835 SOUTH VAN BUREN ST,GREEN BAY,WI,54301,BROWN,(920) 433-0112,Acute Care Hospitals,Voluntary non-profit - Church
